In [36]:
'''
This file reads json file in project/hand_pose/Annotation.json and output project/hand_pose/hand_labels_r270.csv
'''

'\nThis file reads json file in project/hand_pose/Annotation.json and output project/hand_pose/hand_labels_r270.csv\n'

In [37]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import json
from pprint import pprint
from matplotlib import pyplot as plt
from PIL import Image
import sys
import numpy as np
import matplotlib.patches as patches
import cv2
import pandas as pd
import warnings
warnings.simplefilter("ignore")

# Get 2500 images for rotate

In [38]:
base_dir = "../hand_pose"
image_dir = os.path.join(base_dir, 'Color')

In [39]:
img_list = os.listdir(image_dir)
img_name = [img[:-4] for img in img_list]

In [40]:
'''
def rotate(image, theta):
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, theta, 1)
    rotated = cv2.warpAffine(image, M, (int(w),int(h)), cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return rotated
'''

'\ndef rotate(image, theta):\n    (h, w) = image.shape[:2]\n    center = (w / 2, h / 2)\n    M = cv2.getRotationMatrix2D(center, theta, 1)\n    rotated = cv2.warpAffine(image, M, (int(w),int(h)), cv2.INTER_LINEAR,\n                             borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))\n    return rotated\n'

In [41]:
image = cv2.imread(os.path.join(image_dir,img_list[1]))
(h, w) = image.shape[:2]
print(h, w)

1080 1920


In [42]:
img_amount = len(img_list)
print(img_amount)

111


In [43]:
# Following code use tiny dataset for script test

In [44]:
target_dir = os.path.join(base_dir, 'Rotate_270_imgs')
for i in range(5, img_amount, 10):
    img = cv2.imread(os.path.join(image_dir,img_list[i]))
    cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r270.jpg'), np.rot90(img, 3))
    #cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r270.jpg'), rotate(img, 270))

In [45]:
# Following code use real hand_pose dataset with folder "Color" from assignment 1&3

In [46]:
'''
target_dir = os.path.join(base_dir, 'Rotate_270_imgs')
for i in range(5, 56000, 22):
    img = cv2.imread(os.path.join(image_dir,img_list[i]))
    cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r270.jpg'), np.rot90(img, 3))
    #cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r270.jpg'), rotate(img, 270))
'''

"\ntarget_dir = os.path.join(base_dir, 'Rotate_270_imgs')\nfor i in range(5, 56000, 22):\n    img = cv2.imread(os.path.join(image_dir,img_list[i]))\n    cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r270.jpg'), np.rot90(img, 3))\n    #cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r270.jpg'), rotate(img, 270))\n"

# json to jsv for 2500 rotated images

In [47]:
tensorflow_path = "phase1_models"
sys.path.append(tensorflow_path + "/research")
sys.path.append(tensorflow_path + "/research/object_detection")
sys.path.append(tensorflow_path + "/research/object_detection/utils")
from utils import label_map_util
from utils import visualization_utils as vis_util

In [48]:
base_dir = "../hand_pose"
json_file = os.path.join(base_dir, 'annotation.json')
segment_length = 20
csv_file_r270 = os.path.join(base_dir, 'hand_labels_r270.csv')
image_dir_r270 = os.path.join(base_dir, 'Rotate_270_imgs')

In [49]:
img_list_r270 = os.listdir(image_dir_r270)

In [50]:
img_name_r270 = [img[:-9] for img in img_list_r270]

In [51]:
with open(json_file) as f:
    data = json.load(f)
pprint(f)

<_io.TextIOWrapper name='../hand_pose/annotation.json' mode='r' encoding='UTF-8'>


In [52]:
len(data)

94126

In [53]:
print(img_name_r270)

['', '000_2255', '000_2045', '000_221', '000_23', '000_245', '000_2273', '000_2429', '000_2411', '000_299', '000_2027', '000_2183']


In [54]:
len(img_list_r270)

12

In [55]:
def paint_boundary_r270(index):
    img = cv2.imread(os.path.join(image_dir_r270,img_list_r270[index]))
    if img_name_r270[index]+"_L" in data:
        pos = record_boundary_r270(index, "_L")
        img = cv2.rectangle(img, pos[0], pos[1] , (0, 255, 0), 3) #(row['xmin'], row['ymin']), (row['xmax'], row['ymax'])
    if img_name[index]+"_R" in data:
        pos = record_boundary_r270(index, "_R")
        img = cv2.rectangle(img, pos[0], pos[1] , (0, 255, 0), 3) #(row['xmin'], row['ymin']), (row['xmax'], row['ymax'])     
    return img

In [56]:
def record_boundary_r270(index, direction):
    name = img_name_r270[index]+direction
    xy = np.array(data[name])
    x_arr = xy[:,0]
    y_arr = xy[:,1]
    segment_length = 20
    x_max = max(h - y_arr) + segment_length
    y_max = max(x_arr) + segment_length
    x_min = min(h - y_arr) - segment_length
    y_min = min(x_arr) - segment_length
    
    '''
    x_max = max(x_arr) + segment_length
    y_max = max(y_arr) + segment_length
    x_min = min(x_arr) - segment_length
    y_min = min(y_arr) - segment_length
    '''
    return [(int(x_min), int(y_min)), (int(x_max), int(y_max))]

In [57]:
def return_value_r270(index, direction):
    pos = record_boundary_r270(index, direction)
    xmin = pos[0][0]
    ymin = pos[0][1]
    xmax = pos[1][0]
    ymax = pos[1][1]
    value = (img_name_r270[index]+'_r270.jpg', #+direction,
             xmax-xmin,
             ymax-ymin,
             'hand',
             xmin,
             ymin,
             xmax,
             ymax
             )
    return value

In [58]:
xml_list_r270 = []
#for xml_file in glob.glob(path + '/*.xml'):
for index, img in enumerate(img_name_r270):
    if img + '_R' in data:
        xml_list_r270.append(return_value_r270(index, '_R'))
    if img + '_L' in data:
        xml_list_r270.append(return_value_r270(index, '_L'))
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
xml_df_r270 = pd.DataFrame(xml_list_r270, columns=column_name) #use panda
#print(xml_df)

In [59]:
len(xml_list_r270)

15

In [60]:
xml_df_r270.to_csv(csv_file_r270, index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


# Append 5600 images csv to hand_labels.csv

In [61]:
base_dir = "../hand_pose"
csv_file_r270 = os.path.join(base_dir, 'hand_labels_r270.csv')
csv_file = os.path.join(base_dir, 'hand_labels.csv')

In [62]:
labels_r270 = pd.read_csv(csv_file_r270)

In [63]:
labels = pd.read_csv(csv_file)

In [64]:
len(labels_r270)

15

In [65]:
len(labels)

218

In [66]:
results = pd.concat([labels,labels_r270])
results.to_csv(os.path.join(base_dir, 'hand_labels.csv'))

In [67]:
len(results)

233